In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.preprocessing.image
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.metrics
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import skimage.io as io
import skimage.transform as trans
import os;
import datetime  
import cv2 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import matplotlib.gridspec as gridspec
%matplotlib inline

def normalizeImage(img):
    if np.max(img) > 1:
        img = img/255
    return img

def normalizeMask(mask):
    if np.max(mask) > 1:
        mask = mask/255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return mask
    
## load and check data
image_train_arr = []
mask_train_arr = []

#load avaialble training data
for i in range(0,30):
    img = io.imread("../data/membrane/train/image/{}.png".format(i))
    img = normalizeImage(img)
    mask = io.imread("../data/membrane/train/mask/{}.png".format(i))
    mask = normalizeMask(mask)
    image_train_arr.append(img)
    mask_train_arr.append(mask)

xtrainImage = np.array(image_train_arr)
xtrainImage = xtrainImage.reshape(-1, 512, 512, 1)
xtrainMask = np.array(mask_train_arr)
xtrainMask = xtrainMask.reshape(-1, 512, 512, 1)
print(xtrainImage.shape)
print(xtrainMask.shape)

#io.imshow(xtrainImage[1].reshape(512, 512))
#io.imshow(xtrainMask[1].reshape(512, 512))

(30, 512, 512, 1)
(30, 512, 512, 1)
